In [1]:
import os

import torch
import torchvision.transforms as transforms
from PIL import Image

import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder

In [2]:
# for calculating the accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):

    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
# convolution block with BatchNormalization
def ConvBlock(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool2d(4))
    return nn.Sequential(*layers)

In [3]:
# resnet architecture
class CNN_NeuralNet(ImageClassificationBase):
    def __init__(self, in_channels, num_diseases):
        super().__init__()

        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True)
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))

        self.conv3 = ConvBlock(128, 256, pool=True)
        self.conv4 = ConvBlock(256, 512, pool=True)
        #self.conv5 = ConvBlock(256, 256, pool=True)
        #self.conv6 = ConvBlock(256, 512, pool=True)
        #self.conv7 = ConvBlock(512, 512, pool=True)

        self.res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))
        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                       nn.Flatten(),
                                       nn.Linear(512, num_diseases))

    def forward(self, x): # x is the loaded batch
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        #out = self.conv5(out)
        #out = self.conv6(out)
        #out = self.conv7(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [4]:
# for moving data to device (CPU or GPU)
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

# for moving data into GPU (if available)
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available:
        return torch.device("cuda")
    else:
        return torch.device("cpu")

device = get_default_device()

# Step 1: Load the pre-trained model
def load_model(model_path):
    model = torch.load(model_path)
    #model.eval()  # Set the model to evaluation mode
    return model

In [5]:
classes=['Apple___Cedar_apple_rust',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Potato___Early_blight',
 'Tomato___Bacterial_spot',
 'Strawberry___healthy',
 'Tomato___Leaf_Mold',
 'Potato___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Tomato___Early_blight',
 'Peach___healthy',
 'Apple___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Peach___Bacterial_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Grape___Black_rot',
 'Potato___Late_blight',
 'Tomato___Late_blight',
 'Strawberry___Leaf_scorch',
 'Apple___Apple_scab',
 'Blueberry___healthy',
 'Corn_(maize)___Common_rust_',
 'Squash___Powdery_mildew',
 'Raspberry___healthy',
 'Tomato___Septoria_leaf_spot',
 'Soybean___healthy',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___healthy',
 'Apple___Black_rot',
 'Tomato___Target_Spot',
 'Grape___Esca_(Black_Measles)',
 'Cherry_(including_sour)___Powdery_mildew',
 'Tomato___Tomato_mosaic_virus',
 'Pepper,_bell___Bacterial_spot',
 'Grape___healthy',
 'Pepper,_bell___healthy']

In [6]:
model_path = "plants.pth"

model = to_device(CNN_NeuralNet(3, len(classes)), device)

model.load_state_dict(torch.load('plants.pth'))

<All keys matched successfully>

In [7]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize to the input size expected by the model
        transforms.ToTensor(),           # Convert the image to a PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
    ])
    image = Image.open(image_path)
    image = transform(image)
    return image

In [8]:
def predict_image(img, model):
    """Converts image to array and return the predicted class
        with highest probability"""
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label

    return classes[preds[0].item()]

In [9]:
def preprocess_and_predict(folder_path, model):
    # Iterate through each image file in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is an image file
        if filename.endswith(('.jpeg', '.jpg', '.png', '.bmp')):
            # Construct the full path to the image file
            image_path = os.path.join(folder_path, filename)
            image = preprocess_image(image_path)
            # Make predictions using the predict_image function
            prediction = predict_image(image, model)
            # Print the output
            print(f"Image: {filename}, Prediction: {prediction}")


In [10]:
folder_path='../server/images'
preprocess_and_predict(folder_path, model)

Image: 1.jpeg, Prediction: Corn_(maize)___healthy
